In [1]:
# %%
from enum import IntEnum
from pickle import TRUE
from typing import Dict
from tqdm.auto import tqdm
import numpy as np

class ConstellationId(IntEnum):
  # Int values match Ublox gnssid version 8
  GPS = 0
  SBAS = 1
  GALILEO = 2
  BEIDOU = 3
  IMES = 4
  QZNSS = 5
  GLONASS = 6
  # Not supported by Ublox:
  IRNSS = 7

  def to_rinex_char(self) -> str:
    # returns single character id
    return RINEX_CONSTELLATION_TO_ID[self]

  @classmethod
  def from_rinex_char(cls, c: str):
    if c in RINEX_ID_TO_CONSTELLATION:
      return RINEX_ID_TO_CONSTELLATION[c]
    else:
      return None

  @classmethod
  def from_qcom_source(cls, report_source: int):
    if report_source == 0:
      return ConstellationId.GPS
    if report_source == 6:
      return ConstellationId.SBAS
    elif report_source == 1:
      return ConstellationId.GLONASS
    raise NotImplementedError('Only GPS (0), SBAS (1) and GLONASS (6) are supported from qcom, not:', {report_source})
CONSTELLATION_TO_NMEA_RANGES = {
  # NmeaId ranges for each constellation with its svId offset.
  # constellation: [(start, end, svIdOffset)]
  # svId = nmeaId + offset
  ConstellationId.GPS: [(1, 32, 0)],  # svId [1,32]
  ConstellationId.GLONASS: [(33, 59, -32)],  # svId [1,31]
  ConstellationId.GALILEO: [(60, 97, -59)],  # svId 1-36
  ConstellationId.BEIDOU: [(98, 160, -97)],  # svId 1-72
  ConstellationId.SBAS: [(161, 231, -160)],  # svId [1,71]
  ConstellationId.IMES: [(232, 240, -231)],  # svId [1,9]
  ConstellationId.QZNSS: [(241, 268, -240)],  # svId [1,28]  # todo should be QZSS
}
RINEX_CONSTELLATION_TO_ID: Dict[ConstellationId, str] = {
  ConstellationId.GPS: 'G',
  ConstellationId.GLONASS: 'R',
  ConstellationId.SBAS: 'S',
  ConstellationId.GALILEO: 'E',
  ConstellationId.BEIDOU: 'C',
  ConstellationId.QZNSS: 'J',
  ConstellationId.IRNSS: 'I'
}
L1_FREQ: Dict[ConstellationId, float] = {
  ConstellationId.GPS: 1.57542 * 1e9,
  ConstellationId.GLONASS: 1.60200 * 1e9 + 0.5625 * 1e6,
  ConstellationId.SBAS: 1.57542 * 1e9,
  ConstellationId.GALILEO: 1.57542 * 1e9,
  ConstellationId.BEIDOU: 1.561098 * 1e9,
  ConstellationId.QZNSS: 1.57542 * 1e9,
  ConstellationId.IRNSS: 1.57542 * 1e9
}
RINEX_ID_TO_CONSTELLATION: Dict[str, ConstellationId] = {id: con for con, id in RINEX_CONSTELLATION_TO_ID.items()}
def get_constellation(prn: str):
  identifier = prn[0]
  constellation = ConstellationId.from_rinex_char(identifier)
  if constellation is not None:
    return constellation.name
  return None
def get_nmea_id_from_prn(prn: str):
  constellation = get_constellation(prn)
  if constellation is None:
    raise ValueError(f"Constellation not found for prn {prn}")

  sv_id = int(prn[1:])  # satellite id
  return get_nmea_id_from_constellation_and_svid(ConstellationId[constellation], sv_id)

def get_nmea_id_from_constellation_and_svid(constellation: ConstellationId, sv_id: int):
  ranges = CONSTELLATION_TO_NMEA_RANGES[constellation]
  for (start, end, sv_id_offset) in ranges:
    new_nmea_id = sv_id - sv_id_offset
    if start <= new_nmea_id <= end:
      return new_nmea_id

  raise ValueError(f"NMEA ID not found for constellation {constellation.name} with satellite id {sv_id}")



In [2]:
import georinex as gr
base_dat = gr.load("/persist/catkin_ws/src/GraphGNSSLib/global_fusion/dataset/gps_solution_TST/hksc1180.19o")

In [3]:
obs_dat = gr.load("/persist/catkin_ws/src/GraphGNSSLib/global_fusion/dataset/gps_solution_TST/COM3_190428_124409.obs")

In [4]:
# print(dat.coords)
print(obs_dat.coords['sv'])
print(len(obs_dat.coords['time']))
print(obs_dat.coords['time'])

<xarray.DataArray 'sv' (sv: 25)>
array(['C01', 'C02', 'C03', 'C04', 'C05', 'C06', 'C07', 'C08', 'C09', 'C10',
       'C11', 'C13', 'C14', 'C16', 'C23', 'C28', 'G02', 'G04', 'G05', 'G06',
       'G09', 'G12', 'G17', 'G19', 'G25'], dtype='<U3')
Coordinates:
  * sv       (sv) <U3 'C01' 'C02' 'C03' 'C04' 'C05' ... 'G12' 'G17' 'G19' 'G25'
1760
<xarray.DataArray 'time' (time: 1760)>
array(['2019-04-28T12:44:33.996999000', '2019-04-28T12:44:34.996999000',
       '2019-04-28T12:44:35.996999000', ..., '2019-04-28T13:13:51.000999000',
       '2019-04-28T13:13:52.000999000', '2019-04-28T13:13:53.000999000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2019-04-28T12:44:33.996999 ... 2019-04-28T...


In [5]:
nav_dat_n = gr.load("/persist/catkin_ws/src/GraphGNSSLib/global_fusion/dataset/gps_solution_TST/hksc1180.19n")

In [6]:
nav_dat_b = gr.load("/persist/catkin_ws/src/GraphGNSSLib/global_fusion/dataset/gps_solution_TST/hksc1180.19b")

In [7]:
print(nav_dat_b.coords)
print(nav_dat_b.coords['sv'])
print(nav_dat_b.coords['time'])

Coordinates:
  * sv       (sv) <U5 'C01' 'C02' 'C03' 'C04' 'C05' ... 'C27' 'C28' 'C29' 'C30'
  * time     (time) datetime64[ns] 2019-04-19T01:00:00 ... 2019-04-28T23:00:00
<xarray.DataArray 'sv' (sv: 29)>
array(['C01', 'C02', 'C03', 'C04', 'C05', 'C05_1', 'C06', 'C07', 'C08', 'C09',
       'C10', 'C11', 'C12', 'C13', 'C14', 'C16', 'C18', 'C19', 'C20', 'C21',
       'C22', 'C23', 'C24', 'C25', 'C26', 'C27', 'C28', 'C29', 'C30'],
      dtype='<U5')
Coordinates:
  * sv       (sv) <U5 'C01' 'C02' 'C03' 'C04' 'C05' ... 'C27' 'C28' 'C29' 'C30'
<xarray.DataArray 'time' (time: 40)>
array(['2019-04-19T01:00:00.000000000', '2019-04-25T17:00:00.000000000',
       '2019-04-26T09:00:00.000000000', '2019-04-26T17:00:00.000000000',
       '2019-04-27T00:00:00.000000000', '2019-04-27T02:00:00.000000000',
       '2019-04-27T05:00:00.000000000', '2019-04-27T06:00:00.000000000',
       '2019-04-27T07:00:00.000000000', '2019-04-27T09:00:00.000000000',
       '2019-04-27T11:00:00.000000000', '2019-04-27T15

In [8]:
print(nav_dat_b)

<xarray.Dataset>
Dimensions:           (sv: 29, time: 40)
Coordinates:
  * sv                (sv) <U5 'C01' 'C02' 'C03' 'C04' ... 'C28' 'C29' 'C30'
  * time              (time) datetime64[ns] 2019-04-19T01:00:00 ... 2019-04-2...
Data variables: (12/27)
    SVclockBias       (time, sv) float64 nan nan nan nan nan ... nan nan nan nan
    SVclockDrift      (time, sv) float64 nan nan nan nan nan ... nan nan nan nan
    SVclockDriftRate  (time, sv) float64 nan nan nan nan nan ... nan nan nan nan
    AODE              (time, sv) float64 nan nan nan nan nan ... nan nan nan nan
    Crs               (time, sv) float64 nan nan nan nan nan ... nan nan nan nan
    DeltaN            (time, sv) float64 nan nan nan nan nan ... nan nan nan nan
    ...                ...
    SVacc             (time, sv) float64 nan nan nan nan nan ... nan nan nan nan
    SatH1             (time, sv) float64 nan nan nan nan nan ... nan nan nan nan
    TGD1              (time, sv) float64 nan nan nan nan nan ... nan nan

In [9]:
print(nav_dat_n)

<xarray.Dataset>
Dimensions:           (time: 35, sv: 31)
Coordinates:
  * time              (time) datetime64[ns] 2019-04-27T08:00:00 ... 2019-04-29
  * sv                (sv) <U3 'G01' 'G02' 'G03' 'G05' ... 'G30' 'G31' 'G32'
Data variables: (12/28)
    SVclockBias       (time, sv) float64 nan nan nan nan ... nan nan -0.0001112
    SVclockDrift      (time, sv) float64 nan nan nan nan ... nan nan 1.728e-11
    SVclockDriftRate  (time, sv) float64 nan nan nan nan nan ... 0.0 nan nan 0.0
    IODE              (time, sv) float64 nan nan nan nan ... 72.0 nan nan 74.0
    Crs               (time, sv) float64 nan nan nan nan ... 47.53 nan nan 17.59
    DeltaN            (time, sv) float64 nan nan nan nan ... nan nan 4.471e-09
    ...                ...
    L2Pflag           (time, sv) float64 nan nan nan nan nan ... 0.0 nan nan 0.0
    SVacc             (time, sv) float64 nan nan nan nan nan ... 2.0 nan nan 2.0
    health            (time, sv) float64 nan nan nan nan nan ... 0.0 nan nan 0.0


In [10]:
print(nav_dat_n.coords)
print(nav_dat_n.coords['sv'])
print(nav_dat_n.coords['time'])

Coordinates:
  * time     (time) datetime64[ns] 2019-04-27T08:00:00 ... 2019-04-29
  * sv       (sv) <U3 'G01' 'G02' 'G03' 'G05' 'G06' ... 'G29' 'G30' 'G31' 'G32'
<xarray.DataArray 'sv' (sv: 31)>
array(['G01', 'G02', 'G03', 'G05', 'G06', 'G07', 'G08', 'G09', 'G10', 'G11',
       'G12', 'G13', 'G14', 'G15', 'G16', 'G17', 'G18', 'G19', 'G20', 'G21',
       'G22', 'G23', 'G24', 'G25', 'G26', 'G27', 'G28', 'G29', 'G30', 'G31',
       'G32'], dtype='<U3')
Coordinates:
  * sv       (sv) <U3 'G01' 'G02' 'G03' 'G05' 'G06' ... 'G29' 'G30' 'G31' 'G32'
<xarray.DataArray 'time' (time: 35)>
array(['2019-04-27T08:00:00.000000000', '2019-04-27T10:00:00.000000000',
       '2019-04-27T12:00:00.000000000', '2019-04-27T13:59:44.000000000',
       '2019-04-27T14:00:00.000000000', '2019-04-27T16:00:00.000000000',
       '2019-04-27T18:00:00.000000000', '2019-04-27T19:59:44.000000000',
       '2019-04-27T20:00:00.000000000', '2019-04-27T22:00:00.000000000',
       '2019-04-27T23:59:44.000000000', '2019-04-2

In [11]:
Freq = {
    'G1': 1.57542e9,
    'G2': 1.22760e9,
    'G5': 1.17645e9, 
    'R1': 1.602e9 + 0.5625e6,
    'R2': 1.246e9 - 0.4375e6,
    'R3': 1.202025e9,
    'E1': 1.57542e9,
    'E2': 1.22760e9,
    'E5': 1.17645e9, 
    'C1': 1.561098e9,
    'C2': 1.20714e9,
    'C5': 1.26852e9,
    'C7': 1.20714e9,
    'S1': 1.57542e9,
    'S5': 1.17645e9,
}

In [12]:
import numpy as np
def datetime64totimestamp(date64):
    return float((date64 - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's'))

In [13]:
from datetime import datetime
print(nav_dat_b.coords['time'])
print(datetime64totimestamp(nav_dat_b.coords['time'][0].values))
print(nav_dat_n.coords['time'])
print(datetime64totimestamp(nav_dat_n.coords['time'][0].values))

<xarray.DataArray 'time' (time: 40)>
array(['2019-04-19T01:00:00.000000000', '2019-04-25T17:00:00.000000000',
       '2019-04-26T09:00:00.000000000', '2019-04-26T17:00:00.000000000',
       '2019-04-27T00:00:00.000000000', '2019-04-27T02:00:00.000000000',
       '2019-04-27T05:00:00.000000000', '2019-04-27T06:00:00.000000000',
       '2019-04-27T07:00:00.000000000', '2019-04-27T09:00:00.000000000',
       '2019-04-27T11:00:00.000000000', '2019-04-27T15:00:00.000000000',
       '2019-04-27T17:00:00.000000000', '2019-04-27T21:00:00.000000000',
       '2019-04-27T22:00:00.000000000', '2019-04-27T23:00:00.000000000',
       '2019-04-28T00:00:00.000000000', '2019-04-28T01:00:00.000000000',
       '2019-04-28T02:00:00.000000000', '2019-04-28T03:00:00.000000000',
       '2019-04-28T04:00:00.000000000', '2019-04-28T05:00:00.000000000',
       '2019-04-28T06:00:00.000000000', '2019-04-28T07:00:00.000000000',
       '2019-04-28T08:00:00.000000000', '2019-04-28T09:00:00.000000000',
       '2019-0

/tmp/ipykernel_16578/3658927575.py:3: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  return float((date64 - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's'))


In [14]:
code_map = {'I': 1, 'D': 2, 'C': 4, 'Q': 8}

In [15]:
import numpy as np
import os

from pytz import utc

from sensor_msgs.msg import Imu, NavSatFix, Image
from geometry_msgs.msg import PointStamped, PoseStamped
from ninebot_gio_coupling.msg import (
    GnssEphemMsg, 
    GnssGloEphemMsg, 
    GnssMeasMsg, 
    GnssObsMsg, 
    GnssTimeMsg, 
    StampedFloat64Array, 
    GnssPolyEphemMsg, 
    GnssPVMsg,
)
from nav_msgs.msg import Odometry
import sys
sys.path.append('/persist/source_data1/wangyf/dataset/GNSS/Code/laika')
from laika.gps_time import GPSTime, gpst_to_utc, utc_to_gpst, adjweek
import rosbag
import rospy
import joblib
import time
import subprocess
import tqdm
from datetime import datetime, timezone
# rospy.init_node('write_bag')
# set_code = set()
# bag_msg_list = []

# print(set_code)
    

In [37]:
def gen_obs_bag_msg(dat, topic_name, timestamp_lambda=None):
    obs_bag_msg_list = []
    time_array = dat.coords['time']
    for index in range(len(time_array)):
        time_date64 = time_array[index]
        gnss_meas_msg = GnssMeasMsg()
        timestamp = datetime64totimestamp(time_date64.values)
        time_date = datetime.fromtimestamp(timestamp, timezone.utc)
        # print(time_date)
        gps_time_o = GPSTime.from_datetime(time_date)
        gps_time = utc_to_gpst(gps_time_o)
        timestamp = gps_time.as_datetime().timestamp()
        if timestamp_lambda is not None and not timestamp_lambda(timestamp):
            # if timestamp_lambda is not None:
            #     print("time_date64: {} timestamp {}".format(time_date64.values, timestamp))
            continue
        new_dataset = dat.isel(time=index)
        sv_array = new_dataset.coords['sv']
        for j in range(len(sv_array)):
            sv = sv_array[j].values
            data = new_dataset.isel(sv=j)
            # print(len(data.data_vars))
            obs_msg = GnssObsMsg()
            obs_msg.time.week = gps_time.week
            obs_msg.time.tow = gps_time.tow
            obs_msg.sat = get_nmea_id_from_prn(str(sv))
            freq_index = -1
            for ele in sorted(data.data_vars, key=lambda x: x[1:3], reverse=True):
                # set_code.add(ele[2])
                # print(ele, data[ele].values)
                if np.isnan(data[ele].values):
                    continue
                    # print(sv, timestamp, ele, data[ele].values)
                if freq_index != int(ele[1]):
                    freq_index = int(ele[1])
                    # print(str(sv)[0]+ ele[1], Freq)
                    obs_msg.freqs.append(Freq[str(sv)[0]+ele[1]])
                if ele[0] == 'C':
                    obs_msg.psr.append(data[ele].values)
                    obs_msg.psr_std.append(-1)
                elif ele[0] == 'L':
                    obs_msg.cp.append(data[ele].values)
                    obs_msg.cp_std.append(-1)
                elif ele[0] == 'D':
                    obs_msg.dopp.append(data[ele].values)
                    obs_msg.dopp_std.append(-1)
                elif ele[0] == 'S':
                    obs_msg.CN0.append(data[ele].values)
        gnss_meas_msg.meas.append(obs_msg)
        obs_bag_msg_list.append((timestamp, gnss_meas_msg, topic_name))
    return obs_bag_msg_list
    # bag.write('/gnss/meas', gnss_meas_msg, t=rospy.Time.from_sec(timestamp))

In [17]:
obs_bag_msg_list = gen_obs_bag_msg(obs_dat, '/gnss/meas')

/tmp/ipykernel_16578/3658927575.py:3: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  return float((date64 - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's'))


In [32]:
print(len(obs_bag_msg_list))

1760


In [38]:
print(obs_bag_msg_list[0][0], obs_bag_msg_list[-1][0])
base_timestamp_lambda = lambda x : x > obs_bag_msg_list[0][0] - 60 and x < obs_bag_msg_list[-1][0] + 60
# base_timestamp_lambda(1560089800)
base_bag_msg_list = gen_obs_bag_msg(base_dat, '/gnss/base/meas', timestamp_lambda=base_timestamp_lambda)

1556455491.996999 1556457251.000999


/tmp/ipykernel_16578/3658927575.py:3: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  return float((date64 - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's'))


In [39]:
len(base_bag_msg_list)

62

In [ ]:
# from datetime import datetime, timezone
# date_time = datetime.strptime('2019-01-01 00:00:00', '%Y-%m-%d %H:%M:%S')
# print(date_time.timestamp())
# print(date_time.replace(tzinfo=timezone.utc).timestamp())
# print(date_time.astimezone(timezone.utc).timestamp())
# print(datetime.utcnow().astimezone(timezone.utc))
# print(datetime.now())

In [40]:
ura_eph=[         
    2.4,3.4,4.85,6.85,9.65,13.65,24.0,48.0,96.0,192.0,384.0,768.0,1536.0,
    3072.0,6144.0,0.0
];
def uraindex(value):
    i = 0
    while i < 15:
        if (ura_eph[i]>=value):
            break
        i += 1
    return i;

In [51]:
import math
from typing import Optional, List
import xarray
def adjday(t, t0):
    tt= t - t0
    if (tt<-43200.0):
        return t + 86400.0
    if (tt> 43200.0):
        return t - 86400.0
    return t
def gen_nav_bag_msg(nav_dat : Optional[xarray.DataArray], timestamp_lambda=None):
    nav_bag_msg_list = []
    time_array = nav_dat.coords['time']
    for index in range(len(time_array)):
        time_date64 = time_array[index]
        timestamp = datetime64totimestamp(time_date64.values)
        time_date = datetime.fromtimestamp(timestamp, timezone.utc)
        gps_time_o = GPSTime.from_datetime(time_date)
        gps_time = utc_to_gpst(gps_time_o)
        timestamp = gps_time.as_datetime().timestamp()
        if timestamp_lambda is not None and not timestamp_lambda(timestamp):
            continue
        new_dataset = nav_dat.isel(time=index)
        sv_array = new_dataset.coords['sv']
        for j in range(len(sv_array)):
            sv = str(sv_array[j].values)
            # print("sv:", sv)
            data = new_dataset.isel(sv=j)
            # print(len(data.data_vars))
            if sv[0] not in ['C', 'G', 'E', 'J', 'R']:
                continue
            if sv[0] in ['C', 'G', 'E', 'J']:
                if np.isnan(data['SVclockBias'].values):
                    continue
                msg = GnssEphemMsg()
                msg.sat = get_nmea_id_from_prn(sv)
                msg.toc.tow = gps_time.tow
                msg.toc.week = gps_time.week
                msg.af0 = data['SVclockBias'].values
                msg.af1 = data['SVclockDrift'].values
                msg.af2 = data['SVclockDriftRate'].values
                msg.A = data['sqrtA'].values**2
                msg.e = data['Eccentricity'].values
                msg.i0 = data['Io'].values
                msg.OMG0 = data['Omega0'].values
                msg.omg = data['omega'].values
                msg.M0 = data['M0'].values
                msg.delta_n = data['DeltaN'].values
                msg.OMG_dot = data['OmegaDot'].values
                msg.i_dot = data['IDOT'].values
                msg.crc = data['Crc'].values
                msg.crs = data['Crs'].values
                msg.cuc = data['Cuc'].values
                msg.cus = data['Cus'].values
                msg.cic = data['Cic'].values
                msg.cis = data['Cis'].values
                if sv[0] in ['G', 'J']:
                    # print(data['IODE'].values)
                    msg.iode = int(data['IODE'].values)
                    msg.iodc = int(data['IODC'].values)
                    msg.toe_tow = data['Toe'].values
                    msg.week = int(data['GPSWeek'].values)
                    msg.code = int(data['CodesL2'].values)
                    msg.health = int(data['health'].values)
                    # msg.flag = data['L2Pflag'].values
                    msg.tgd0 = data['TGD'].values
                    # msg.sva = uraindex(data['SVacc'].values)
                    gps_time1 = adjweek(GPSTime(gps_time.week, data['Toe'].values), gps_time)
                    gps_time2 = adjweek(GPSTime(gps_time.week, data['TransTime'].values), gps_time)
                    msg.toe.week = gps_time1.week
                    msg.toe.tow = gps_time1.tow
                    msg.ttr.week = gps_time2.week
                    msg.ttr.tow = gps_time2.tow
                elif sv[0] == 'E':
                    msg.week = int(data['GALWeek'].values)
                    gps_time1 = adjweek(GPSTime(gps_time.week, data['Toe'].values), gps_time)
                    gps_time2 = adjweek(GPSTime(gps_time.week, data['TransTime'].values), gps_time)
                    msg.toe.week = gps_time1.week
                    msg.toe.tow = gps_time1.tow
                    msg.ttr.week = gps_time2.week
                    msg.ttr.tow = gps_time2.tow
                    msg.health = int(data['health'].values)
                    msg.iode = int(data['IODnav'].values)
                    msg.toe_tow = data['Toe'].values
                    msg.code = int(data['DataSrc'].values)
                    msg.tgd0 = data['BGDe5a'].values
                    msg.tgd1 = data['BGDe5b'].values
                elif sv[0] == 'C':
                    gps_time = gps_time + 14
                    msg.week = int(data['BDTWeek'].values)
                    gps_time1 = adjweek(GPSTime(gps_time.week, data['Toe'].values) + 14, gps_time)
                    gps_time2 = adjweek(GPSTime(gps_time.week, data['TransTime'].values) + 14, gps_time)
                    msg.toe.week = gps_time1.week
                    msg.toe.tow = gps_time1.tow
                    msg.ttr.week = gps_time2.week
                    msg.ttr.tow = gps_time2.tow    
                    msg.health = int(data['SatH1'].values)
                    msg.tgd0 = data['TGD1'].values
                    msg.tgd1 = data['TGD2'].values
                    msg.iodc = int(data['AODE'].values)
                    msg.iode = int(data['AODC'].values)
                    msg.toe_tow = data['Toe'].values
                nav_bag_msg_list.append((timestamp, msg, "/gnss/ephem"))
            if sv[0] == 'R':
                if np.isnan(data['health'].values):
                    continue
                msg = GnssGloEphemMsg()
                msg.sat = get_nmea_id_from_prn(sv)
                dow=math.floor(gps_time.tow/86400.0)
                new_tow = math.floor((gps_time.tow+450.0)/900.0)*900
                gps_time1 = utc_to_gpst(GPSTime(gps_time.week, new_tow))
                msg.toe.week = gps_time1.week
                msg.toe.tow = gps_time1.tow
                tod = math.fmod(data['MessageFrameTime'].values,86400.0) + dow*86400.0
                tof = utc_to_gpst(adjday(GPSTime(gps_time.week, tod), gps_time1))
                msg.ttr.week = tof.week
                msg.ttr.tow = tof.tow
                msg.iode = int(math.fmod(gps_time.tow+10800.0,86400.0)/900.0+0.5)
                msg.tau_n = data['SVclockBias'].values
                msg.gamma = data['SVrelFreqBias'].values
                msg.pos_x = data['X'].values
                msg.pos_y = data['Y'].values
                msg.pos_z = data['Z'].values
                msg.vel_x = data['dX'].values
                msg.vel_y = data['dY'].values
                msg.vel_z = data['dZ'].values
                msg.acc_x = data['dX2'].values
                msg.acc_y = data['dY2'].values
                msg.acc_z = data['dZ2'].values
                msg.health = int(data['health'].values)
                msg.freqo = int(data['FreqNum'].values)
                msg.age = int(data['AgeOpInfo'].values)
                nav_bag_msg_list.append((timestamp, msg, "/gnss/gloephem"))
    return nav_bag_msg_list
            # bag_msg_list.append((timestamp, gnss_meas_msg, '/gnss/meas'))

In [42]:
b_nav_msg_list = gen_nav_bag_msg(nav_dat_b)

/tmp/ipykernel_16578/3658927575.py:3: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  return float((date64 - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's'))


In [43]:
n_nav_msg_list = gen_nav_bag_msg(nav_dat_n)

/tmp/ipykernel_16578/3658927575.py:3: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  return float((date64 - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's'))


In [44]:
bag_msg_list = obs_bag_msg_list + base_bag_msg_list
bag_msg_list.sort(key=lambda x: x[0])
print("{} -> {}".format(bag_msg_list[0][0], bag_msg_list[-1][0]))

1556455458.0 -> 1556457288.0


In [53]:
all_nav_msg_list_tmp = b_nav_msg_list + n_nav_msg_list
all_nav_msg_list = []
for timestamp, bag_msg, topic_name in all_nav_msg_list_tmp:
    new_timestamp = timestamp - 7200
    if new_timestamp < bag_msg_list[0][0] - 5:
        new_timestamp = bag_msg_list[0][0] - 5
    if new_timestamp > bag_msg_list[-1][0] + 5:
        continue
    all_nav_msg_list.append((new_timestamp, bag_msg, topic_name))
all_nav_msg_list_tmp = []

In [54]:
print("{}->{}".format(all_nav_msg_list[0][0], all_nav_msg_list[-1][0]))

1556455453.0->1556455453.0


In [55]:
all_bag_msg_list = bag_msg_list + all_nav_msg_list
all_bag_msg_list.sort(key=lambda x: x[0])

In [56]:
nav_dat_b.attrs

{'ionospheric_corr_BDS': array([ 9.3132e-09,  8.9407e-08, -1.0133e-06,  2.0862e-06,  1.2493e+05,
        -6.8813e+05,  6.8813e+06, -7.4056e+06]),
 'version': 3.02,
 'svtype': ['C'],
 'rinextype': 'nav',
 'filename': 'hksc1180.19b'}

In [28]:
nav_dat_n.attrs

{'ionospheric_corr_GPS': array([ 9.3132e-09,  1.4901e-08, -5.9605e-08, -1.1921e-07,  8.8064e+04,
         4.9152e+04, -1.3107e+05, -3.2768e+05]),
 'version': 3.02,
 'svtype': ['G'],
 'rinextype': 'nav',
 'filename': 'hksc1180.19n'}

In [48]:
nav_dat_n.attrs['ionospheric_corr_GPS']

array([ 9.3132e-09,  1.4901e-08, -5.9605e-08, -1.1921e-07,  8.8064e+04,
        4.9152e+04, -1.3107e+05, -3.2768e+05])

In [57]:
iono_msg = StampedFloat64Array()
iono_msg.data = nav_dat_n.attrs['ionospheric_corr_GPS']
iono_msg.header.stamp = rospy.Time.from_sec(all_bag_msg_list[0][0])


In [58]:
bag = rosbag.Bag('test.bag', 'w')
for timestamp, bag_msg, topic_name in all_bag_msg_list:
    # print(timestamp, bag_msg, topic_name)
    bag.write(topic_name, bag_msg, t=rospy.Time.from_sec(timestamp))
bag.write('/gnss/iono', iono_msg, t=iono_msg.header.stamp)
bag.close()